# DNBLab Jupyter Notebook Tutorial

## Audioanalyse - Schnittstellenabfrage, Ausgabe und Visualisierungen

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage zu einem Audioobjekt über die SRU-Schnittstelle sowie die WAV-Ausgabe und eine Auswahl von Visualisierungen. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Anfrage und Ausgabe der Daten in MARC21-xml zur weiteren Verarbeitung.

Die Daten können über die SRU-Schnittstelle als XML-Antwort in folgenden Formaten ausgeliefert werden: 
* MARC21-xml http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd
* DNB Casual (oai_dc)	http://www.openarchives.org/OAI/2.0/oai_dc.xsd
* RDF (RDFxml) http://www.w3.org/2000/07/rdf.xsd

## 1. Einrichten der Arbeitsumgebung <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die SRU-Schnittstelle wird BeautifulSoup https://www.crummy.com/software/BeautifulSoup/ und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit Pandas https://pandas.pydata.org/ können Elemente aus dem MARC21-Format ausgelesen werden.

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## 2. SRU-Abfrage mit Ausgabe in MARC21-xml<a class="anchor" id="Teil2"></a>

Die Funktion dnb_sru nimmt den Paramter "query" der SRU-Abfrage entgegen und liefert alle Ergebnisse als eine Liste von records aus.

In [ ]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## 3. Durchsuchen eines MARC-Feldes<a class="anchor" id="Teil3"></a>

Die Funktion parse_records nimmt als Parameter jeweils ein Record entgegen und sucht über xpath die gewünschte Informationen (in diesem Fall IDN und Titel) heraus und liefert diese als dictionary zurück. 

In [ ]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #umfang = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
         # urn
    urn = xml.xpath("marc:datafield[@ind1 = ' ']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        link = link[0].text
        #link = unicodedata.normalize("NFC", urn)
    except:
        link = 'fail'
        
         # umfang
    umfang = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        umfang = umfang[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        umfang = 'fail'
        
    meta_dict = {"idn":idn,
                 "titel":titel,
                 "link":link,
                 "umfang":umfang
                               }
    
    return meta_dict

Die verschiedenen Indexbezeichnungen stehen in https://services.dnb.de/sru/dnb?operation=explain&version=1.1 und können mittels CQL gezielt nach z.B. dem Titelstichwort "Klimawandel" in Kombination mit Standort "online frei verfügbar" abgefragt werden. Auf diese Art kann durch Anpassen des Codes auch nach anderen Begriffen oder Namen in beliebigen MARC-Feldern gesucht werden. 

In [ ]:
records = dnb_sru('tit=Fischermädchen  and cod=t003 and location=onlinefree')
print(len(records), 'Ergebnisse')

Mit print() werden alle MARC-Felder der gefundenen Datensätze ausgegeben.

In [ ]:
print(records)

## 4. Anzeige zur weiteren Bearbeitung <a class="anchor" id="Teil4"></a>

Mit der "Python Data Analysis Library" Pandas für Python werden die Ergebnisse als Dataframe ausgegeben. Die ersten 5 und letzten 5 Zeilen des Dataframes können mit dem Befehl "df = pd.DataFrame() angezeigt werden. Dabei können in ein Set Objekte beliebigen Datentyps gespeichert werden. 

In [ ]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

## 5. Download der WAV-Datei <a class="anchor" id="Teil5"></a>

Mit der Funktion wget können die Dateien des ermittelten Archivobjekt-Links heruntergeladen und mit unzip im Verzeichnis der aktuellen Jupyter Sitzung gespeichert werden.

In [ ]:
import wget

!wget https://d-nb.info/1076785042/34

In [ ]:
!unzip 34

## 6. Laden und Abspielen der WAV-Datei <a class="anchor" id="Teil6"></a>

Mit der freien python Bibliothek essentia kann die entpackte WAV-Datei geladen und gestreamed werden.

In [ ]:
import essentia

# Submodule:
import essentia.standard
import essentia.streaming

# Ausgabe des enthaltenen Standard-Angebots
print(dir(essentia.standard))


In [ ]:
# Laden der WAV-Datei
loader = essentia.standard.MonoLoader(filename='1111501032304_00020_00020_01_o.wav')
audio = loader()

In [ ]:
# Abspielen der WAV-Datei - dauert einen Moment
import IPython
IPython.display.Audio('1111501032304_00020_00020_01_o.wav')

## 7. Laden und Abspielen der WAV-Datei <a class="anchor" id="Teil7"></a>

Mit den Bibliotheken pylab und matplotlib wird Sekunde 1 bis 2 der geladenen WAV-Datei geplottet.

In [ ]:
from pylab import plot, show, figure, imshow
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 6) # Anpassen der Plot Größen > als der Stabdardwertset lt

plot(audio[1*44100:2*44100])
plt.title("Visualisierung der 2. Sekunde:")
show() 

## 7. Anzeige von Spektrum, Mel Band Energien und MFCCs <a class="anchor" id="Teil8"></a>

Mit den Bibliotheken pylab und matplotlib wird Sekunde 1 bis 2 der geladenen WAV-Datei geplottet.

Laden des Magnitude Spektrum aus essentia.standard in einem Hann Fenster.

In [ ]:
from essentia.standard import *
w = Windowing(type = 'hann')
spectrum = Spectrum()  # FFT() would return the complex FFT, here we just want the magnitude spectrum
mfcc = MFCC()

Anzeige Spektrum, Mel Bans Energien nd MFCCs in einem Hann Fenster. 

In [ ]:
frame = audio[6*44100 : 6*44100 + 1024]
spec = spectrum(w(frame))
mfcc_bands, mfcc_coeffs = mfcc(spec)

plot(spec)
plt.title("Das Spektrum eines Fensters:")
show()

plot(mfcc_bands)
plt.title("Mel Band Spektral Energien eines Fensters:")
show()

plot(mfcc_coeffs)
plt.title("Die ersten 13 MFCCs eines Fensters:")
show()